In [1]:
import numpy as np
import pandas as pd
import numpy as np
from itertools import cycle
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_curve, f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
#load training data
data = pd.read_csv('train.csv')
data.head(5)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [3]:
# Create dummy variables for every unique category in Division
le = LabelEncoder()
# data["Division"] = le.fit_transform(data["Division"])
# data["Gender"] = le.fit_transform(data["Gender"])
# data["Channel_of_Recruitment"] = le.fit_transform(data["Channel_of_Recruitment"])
# data["Foreign_schooled"] = le.fit_transform(data["Foreign_schooled"])
# data["Marital_Status"] = le.fit_transform(data["Marital_Status"])
# data["Past_Disciplinary_Action"] = le.fit_transform(data["Past_Disciplinary_Action"])
# data["State_Of_Origin"] = le.fit_transform(data["State_Of_Origin"])
# data["Previous_IntraDepartmental_Movement"] = le.fit_transform(data["Previous_IntraDepartmental_Movement"])
data['Qualification'] = data['Qualification'].fillna('First Degree or HND')
# data['Qualification'] = le.fit_transform(data['Qualification'])
data['No_of_previous_employers'] = data['No_of_previous_employers'].replace('More than 5', 6)
data['No_of_previous_employers'] = pd.to_numeric(data['No_of_previous_employers'])
data.drop(['EmployeeNo'], axis=1, inplace=True)

In [4]:
numericals = data[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']]
numericals.head()

,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,No_of_previous_employers
0,2,12.5,1,0,41,0
1,2,12.5,0,0,52,0
2,2,7.5,0,0,42,0
3,3,2.5,0,0,42,1
4,3,7.5,0,0,77,1


In [5]:
categorical = data[['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action']]
categorical.head()

,Division,Qualification,Gender,Channel_of_Recruitment,State_Of_Origin,Foreign_schooled,Marital_Status,Previous_IntraDepartmental_Movement,Past_Disciplinary_Action
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,ANAMBRA,No,Married,No,No
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,ANAMBRA,Yes,Married,No,No
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,KATSINA,Yes,Married,No,No
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,NIGER,Yes,Single,No,No
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,AKWA IBOM,Yes,Married,No,No


In [6]:
data[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']] = MinMaxScaler().fit_transform(numericals)
data.head(5)

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,0.000000,1986,1.0,2011,1.0,0.0,0.166667,ANAMBRA,No,Married,No,No,0.000000,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,0.000000,1991,1.0,2015,0.0,0.0,0.350000,ANAMBRA,Yes,Married,No,No,0.000000,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,0.000000,1987,0.6,2012,0.0,0.0,0.183333,KATSINA,Yes,Married,No,No,0.000000,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,0.111111,1982,0.2,2009,0.0,0.0,0.183333,NIGER,Yes,Single,No,No,0.166667,0
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,0.111111,1990,0.6,2012,0.0,0.0,0.766667,AKWA IBOM,Yes,Married,No,No,0.166667,0


In [7]:
cat_cols = pd.get_dummies(categorical, columns = ['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action'])
data = pd.concat([data, cat_cols], axis = 1)
data.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,...,State_Of_Origin_ZAMFARA,Foreign_schooled_No,Foreign_schooled_Yes,Marital_Status_Married,Marital_Status_Not_Sure,Marital_Status_Single,Previous_IntraDepartmental_Movement_No,Previous_IntraDepartmental_Movement_Yes,Past_Disciplinary_Action_No,Past_Disciplinary_Action_Yes
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,0.000000,1986,1.0,2011,1.0,0.0,...,0,1,0,1,0,0,1,0,1,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,0.000000,1991,1.0,2015,0.0,0.0,...,0,0,1,1,0,0,1,0,1,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,0.000000,1987,0.6,2012,0.0,0.0,...,0,0,1,1,0,0,1,0,1,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,0.111111,1982,0.2,2009,0.0,0.0,...,0,0,1,0,0,1,1,0,1,0
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,0.111111,1990,0.6,2012,0.0,0.0,...,0,0,1,1,0,0,1,0,1,0


In [8]:
data.drop(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action'], axis = 1, inplace = True)
data.head()

,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,No_of_previous_employers,Promoted_or_Not,Division_Business Finance Operations,...,State_Of_Origin_ZAMFARA,Foreign_schooled_No,Foreign_schooled_Yes,Marital_Status_Married,Marital_Status_Not_Sure,Marital_Status_Single,Previous_IntraDepartmental_Movement_No,Previous_IntraDepartmental_Movement_Yes,Past_Disciplinary_Action_No,Past_Disciplinary_Action_Yes
0,0.000000,1986,1.0,2011,1.0,0.0,0.166667,0.000000,0,0,...,0,1,0,1,0,0,1,0,1,0
1,0.000000,1991,1.0,2015,0.0,0.0,0.350000,0.000000,0,0,...,0,0,1,1,0,0,1,0,1,0
2,0.000000,1987,0.6,2012,0.0,0.0,0.183333,0.000000,0,0,...,0,0,1,1,0,0,1,0,1,0
3,0.111111,1982,0.2,2009,0.0,0.0,0.183333,0.166667,0,0,...,0,0,1,0,0,1,1,0,1,0
4,0.111111,1990,0.6,2012,0.0,0.0,0.766667,0.166667,0,0,...,0,0,1,1,0,0,1,0,1,0


In [9]:
#split training dataset into train and test
y = data['Promoted_or_Not']
X = data.drop('Promoted_or_Not', axis=1)

SelectKBest(chi2, k=40).fit_transform(X, y)

In [10]:
# get class distribution
print ("Not Promoted:", data['Promoted_or_Not'][data['Promoted_or_Not'] == 0].count()) #class = 0
print ("Promoted:", data['Promoted_or_Not'][data['Promoted_or_Not'] == 1].count()) #class = 1

Not Promoted: 35071
Promoted: 3241


In [11]:
# Oversampling the dataset
sm = ADASYN(random_state=1)
X, y = sm.fit_sample(X, y)
    
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

my_XGB_model = XGBRegressor(nthread=-1,objective='binary:logistic',missing=None,seed=8)
grid = RandomizedSearchCV(my_XGB_model, params, n_jobs=1,n_iter=10)
grid.fit(X_train, y_train, verbose=True)

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=-1,
                                          objective='binary:logistic',
                                          r...
                   iid='warn', n_iter=10, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                        

In [12]:
XGB_predictions = grid.predict(X_test)
XGB_predictions = [round(value) for value in XGB_predictions]
score = f1_score(y_test, XGB_predictions)
print(score)

0.9677993767621308


In [13]:
#load and predict using the test dataset provided
test_id = pd.read_csv('test.csv')
test = pd.read_csv('test.csv')


# test["Division"] = le.fit_transform(test["Division"])
# test["Gender"] = le.fit_transform(test["Gender"])
# test["Channel_of_Recruitment"] = le.fit_transform(test["Channel_of_Recruitment"])
# test["Foreign_schooled"] = le.fit_transform(test["Foreign_schooled"])
# test["Marital_Status"] = le.fit_transform(test["Marital_Status"])
# test["Past_Disciplinary_Action"] = le.fit_transform(test["Past_Disciplinary_Action"])
# test["State_Of_Origin"] = le.fit_transform(test["State_Of_Origin"])
# test["Previous_IntraDepartmental_Movement"] = le.fit_transform(test["Previous_IntraDepartmental_Movement"])
test['Qualification'] = test['Qualification'].fillna('First Degree or HND')
# test['Qualification'] = le.fit_transform(test['Qualification'])
test['No_of_previous_employers'] = test['No_of_previous_employers'].replace('More than 5', 6)
test['No_of_previous_employers'] = pd.to_numeric(test['No_of_previous_employers'])
test.drop(['EmployeeNo'], axis=1, inplace=True)

In [14]:
numericals = test[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']]
test[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']] = MinMaxScaler().fit_transform(numericals)
categorical = test[['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action']]
cat_cols = pd.get_dummies(categorical, columns = ['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action'])
test = pd.concat([test, cat_cols], axis = 1)
test.drop(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment', 'State_Of_Origin', 'Foreign_schooled', 'Marital_Status', 'Previous_IntraDepartmental_Movement', 'Past_Disciplinary_Action'], axis = 1, inplace = True)
test.head()

,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,No_of_previous_employers,Division_Business Finance Operations,Division_Commercial Sales and Marketing,...,State_Of_Origin_ZAMFARA,Foreign_schooled_No,Foreign_schooled_Yes,Marital_Status_Married,Marital_Status_Not_Sure,Marital_Status_Single,Previous_IntraDepartmental_Movement_No,Previous_IntraDepartmental_Movement_Yes,Past_Disciplinary_Action_No,Past_Disciplinary_Action_Yes
0,0.0,1976,0.6,2017,0.0,0.0,0.566667,0.166667,0,0,...,0,0,1,1,0,0,1,0,1,0
1,0.0,1991,0.0,2018,0.0,0.0,0.633333,0.166667,0,0,...,0,0,1,1,0,0,1,0,1,0
2,0.0,1984,0.6,2012,0.0,0.0,0.750000,0.166667,0,0,...,0,0,1,1,0,0,1,0,1,0
3,0.0,1984,0.2,2009,0.0,0.0,0.350000,0.166667,0,0,...,0,1,0,0,0,1,1,0,1,0
4,0.0,1983,0.6,2014,1.0,0.0,0.633333,0.000000,0,0,...,0,0,1,1,0,0,1,0,1,0


In [15]:
pred = grid.predict(test.values)
pred = [int(round(value)) for value in pred]

output = pd.DataFrame({'EmployeeNo': test_id.EmployeeNo, 'Promoted_or_Not': pred})
output.to_csv('staffCVsubmission.csv', index=False)